In [1]:
import os

In [2]:
%pwd

'c:\\ds_projects\\End-to-End-Lung-Cancer-Classification-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\ds_projects\\End-to-End-Lung-Cancer-Classification-MLflow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path
    testing_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from lung_cancer.constants import *
from lung_cancer.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root]) # create artifacts directory


    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath) # return artifacts/prepare_callbacks/checkpoint_dir
        create_directories([
            Path(model_ckpt_dir), # artifacts/prepare_callbacks/checkpoint_dir
            Path(config.tensorboard_root_log_dir) # create artifacts/prepare_callbacks/tensorboard_log_dir
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data","train")
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data","valid")
        testing_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data","test")
        create_directories([
            Path(training.root_dir) # Create artifacts/training 
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            validation_data=Path(validation_data),
            testing_data=Path(testing_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        self.early_stopping = tf.keras.callbacks.EarlyStopping(monitor= 'val_loss', patience= 15)


    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self.early_stopping
        ]

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:
#gpus = len(tf.config.experimental.list_physical_devices('GPU'))>0
#gpus

True

In [13]:
#import tensorflow as tf
#gpus = tf.config.experimental.list_physical_devices('GPU')
#if gpus:
#    try:
#        for gpu in gpus:
#            tf.config.experimental.set_memory_growth(gpu, True)
#    except RuntimeError as e:
#        print(e)

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    # load model
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    
    def train_valid_test_generator(self):

        # kwargs for datageerator
        datagenerator_kwargs = dict(
            dtype='float32',
            rescale = 1./255,
           # validation_split=0.20
        )
         # kwargs for dataflow
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        # validation data set(datagenerator and generator)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            #subset="validation",
            shuffle=False,
            class_mode="categorical",
            **dataflow_kwargs
        )
         # training data set(datagenerator and generator)
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            #subset="training",
            shuffle=True,
            class_mode="categorical",
            **dataflow_kwargs
        )
        # testing data set(datagenerator and generator)
        test_datagenerator  =  tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.testing_data,
            shuffle=False,
            class_mode="categorical"
            )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_test_generator()
    training.train(callback_list=callback_list)
    
except Exception as e:
    raise e

[2023-10-10 15:34:42,798: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-10 15:34:42,801: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-10 15:34:42,802: INFO: common: created directory at: artifacts]
[2023-10-10 15:34:42,803: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-10-10 15:34:42,804: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-10-10 15:34:42,807: INFO: common: created directory at: artifacts\training]
Found 72 images belonging to 4 classes.
Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
Epoch 1/50
38/38 [==============================] - 55s 1s/step - loss: 1.3646 - accuracy: 0.3936 - val_loss: 1.1603 - val_accuracy: 0.4531
Epoch 2/50


c:\Users\eslam\anaconda3\envs\cnn-cancer-env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


38/38 [==============================] - 61s 2s/step - loss: 1.0125 - accuracy: 0.5745 - val_loss: 1.0211 - val_accuracy: 0.6406
Epoch 3/50
38/38 [==============================] - 58s 2s/step - loss: 0.9486 - accuracy: 0.5980 - val_loss: 0.9693 - val_accuracy: 0.5312
Epoch 4/50
38/38 [==============================] - 58s 2s/step - loss: 0.8615 - accuracy: 0.6214 - val_loss: 0.9178 - val_accuracy: 0.5156
Epoch 5/50
38/38 [==============================] - 58s 2s/step - loss: 0.8093 - accuracy: 0.6801 - val_loss: 0.8481 - val_accuracy: 0.6250
Epoch 6/50
38/38 [==============================] - 58s 2s/step - loss: 0.7854 - accuracy: 0.6600 - val_loss: 0.8275 - val_accuracy: 0.6094
Epoch 7/50
38/38 [==============================] - 58s 2s/step - loss: 0.7413 - accuracy: 0.6750 - val_loss: 0.8612 - val_accuracy: 0.5938
Epoch 8/50
38/38 [==============================] - 61s 2s/step - loss: 0.7238 - accuracy: 0.6884 - val_loss: 0.8396 - val_accuracy: 0.6719
Epoch 9/50
38/38 [=============